# モデルファイル
最終的にはpyファイルにして、引数でデータのバージョン管理ができるようにすること

In [1]:
import os, sys, pdb
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import gc
%matplotlib inline
%load_ext autoreload
%autoreload 1
DATA_DIR = Path('../input')
MODEL_DIR = Path('../model')
OUT_DIR = Path('../output')
import warnings
warnings.filterwarnings('ignore')

In [2]:
from __future__ import print_function, division
import torch
from torch import nn
import torchvision
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from random import shuffle
import random
random.seed(seed)
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms, utils
plt.ion()   # interactive mode

In [5]:
supported_rnns = {
    'lstm': nn.LSTM,
    'rnn': nn.RNN,
    'gru': nn.GRU
}
supported_rnns_inv = dict((v, k) for k, v in supported_rnns.items())

In [2]:
from eeglibrary import EEG, EEGDataSet, EEGDataLoader, EEGParser
